In [1]:
%load_ext autoreload
%autoreload 2
from torch.utils.data import Dataset, DataLoader
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LinearRegression
import torch, copy
import torch.nn as nn
import torch.optim as optim
import torch.utils.data as Data
import torch.nn.functional as F
import torchvision
import  matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import gc, os
from module.model import TwoLayerNet
from module.Weight_tune import *
from module.Reorg import *
from utils import validate, check_acceptable
from module.Cram import *


# New Learning mechanism

### Data

In [2]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

data = pd.read_csv('Copper_forecasting_data.csv')

sc = StandardScaler()
X = data.drop(["y"], axis = 1)
X = sc.fit_transform(X)
y = data["y"] / 1000

train_size = int(len(X)*0.8)
batch_size = 30

X_train = X[:train_size, :]
y_train = y[:train_size]
X_test = X[train_size:, :]
y_test = y[train_size:] 

X_train = torch.tensor(np.array(X_train), dtype=torch.float32)
X_test = torch.tensor(np.array(X_test), dtype=torch.float32)
y_train = torch.tensor(np.array(y_train), dtype=torch.float32)
y_test = torch.tensor(np.array(y_test), dtype=torch.float32)
input_dim = X_train.shape[1]

class MyDataset(Data.Dataset):
    def __init__(self, X, y):

        self.X = X
        self.y = y

    def __getitem__(self, index):
        X_, y_ = self.X[index], self.y[index]
        return X_, y_

    def __len__(self):
        return len(self.X)
    

test_loader = torch.utils.data.DataLoader(
    MyDataset(X_test.to(device), y_test.to(device)), 
    batch_size = batch_size, 
    shuffle=False, 
    drop_last = False)


### Model

In [3]:
model = TwoLayerNet(input_dim, 1, 1)

### Config

In [4]:
# learning goal: max eps < learning goal
learning_goal = torch.sqrt(torch.tensor(.09, dtype=torch.float32)) 
lr_rate = .01
lr_bound = 1e-5
criterion = nn.MSELoss()
epochs = 100
"""
# Note
1. hidden dim should check the previous model. dynamically change
2. For lr_rate, lr bound, lr goal (eps bound) are all the same fro eahc module
"""
config_wt = {
    "epochs": epochs,
    "criterion": criterion,        # loss function
    "lr_rate": lr_rate,            # learning rate 
    "lr_bound": lr_bound,          # lower bound of learning rate 
    "lr_goal": learning_goal,      # if regular eps < eps_reg: accept the model
}

config_cram = {
    "lr_goal": learning_goal, 
    "s": 0.01,                     # a small num in cram
}

config_reorg  = {
    "epochs": epochs,
    "criterion": criterion,        # loss function
    "lr_rate": lr_rate,            # learning rate 
    "lr_bound": lr_bound,          # lower bound of learning rate 
    "lr_goal": learning_goal,      # if regular eps < eps_reg: accept the model
    "print_reg": False,            # print detail, eg. loss for each epoch, or not
    "print_w_tune": False,         # print detail, eg. loss for each epoch, or not
    "validate_run": False,         # validate the model, or not
}
# NOTE 
# 1. for leaning goals, if first using weightune and no LTS or otherthings, 
# 13 for getting acceptable wt | 10 not acceptable


### Full Path

- L11 p9, third learning mechanism

In [42]:
# 1. initializing_1_ReLU_LR | L11 p2
# Get weight by ols and set weight of two layers by the weight
def init_model(X_train, y_train):
    miny = min(y_train)
    model = LinearRegression()
    model.fit(X_train, (y_train - miny))
    w = torch.tensor(model.coef_, dtype=torch.float32).reshape(1, -1)
    b = torch.tensor(model.intercept_, dtype=torch.float32).reshape(1)

    model = TwoLayerNet(X_train.shape[1], 1, 1)
    model.layer_1.weight.data = w
    model.layer_1.bias.data = b
    model.layer_out.weight.data = torch.tensor(1, dtype=torch.float32).reshape(1, 1)
    model.layer_out.bias.data = miny.reshape(1)
    return model


# 2. obtaining_LTS / selecting_LTS
def lts(model, X_train, y_train, lr_goal):
    """
    X_train, y_trian, lr_goal
    ---
    # output:
    train_loader: with lts
    n of train size 
    """
    device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

    # predict and residuals
    y_pred = model(X_train)
    y_train = y_train.reshape(-1 ,1)
    resid_square = torch.square(y_pred - y_train).reshape(-1)

    # obtaining
    # prompt: find the indices of tensor < k only shape 1 tensor
    resid_square, sorted_indices = torch.sort(resid_square) # default ascending
    indices_lts = sorted_indices[resid_square < lr_goal**2]
    X_train_lts, y_train_lts = X_train[indices_lts], y_train[indices_lts]

    # check if obtaining is true. 0 is correct
    print(f"Total obtaining n: {len(indices_lts)}")
    print(f"obtaining n over lr goal: {(torch.square(model(X_train_lts) - y_train_lts) > lr_goal**2).sum()}")

    # selecting
    n = len(indices_lts) + 1
    indices_lts = sorted_indices[:n]
    X_train_lts, y_train_lts = X_train[indices_lts], y_train[indices_lts]

    # check if the selected is true. 1 is correct
    print(f"Total select n: {len(indices_lts)}")
    print(f"select n over lr goal: {(torch.square(model(X_train_lts) - y_train_lts)>lr_goal**2).sum()}")

    
    # to tensor 
    X_train_lts = torch.tensor(np.array(X_train_lts), dtype=torch.float32)
    y_train_lts = torch.tensor(np.array(y_train_lts), dtype=torch.float32)
    batch_size = 30

    # train loader of lts
    train_loader = torch.utils.data.DataLoader(
        MyDataset(X_train_lts.to(device), y_train_lts.to(device)), 
        batch_size = batch_size, 
        shuffle=False, 
        drop_last = False)

    return train_loader, [indices_lts[-1]], len(X_train_lts)

model = init_model(X_train, y_train)
lts(model, X_train, y_train, learning_goal)

Total obtaining n: 64
obtaining n over lr goal: 0
tensor([  9, 330, 352, 110, 290, 302, 323, 319,  85, 214, 192, 271, 325, 332,
        372, 236, 324, 322, 333, 102, 199,  92, 114, 139, 131, 358,  73, 112,
        288, 137, 357, 212, 148, 165, 184, 237,  75,  97, 362, 159, 101, 268,
         33,  16, 318, 147, 227, 141, 142, 127,  98, 215, 373,  19, 218, 138,
        155, 213, 361, 111,   3, 140,  11,  21,  44])
Total select n: 65
select n over lr goal: 1


(<torch.utils.data.dataloader.DataLoader at 0x20ba1f56e30>, [tensor(44)], 65)

In [91]:
model = init_model(X_train, y_train)
n = 0
while n < 2000:
    train_loader, indices_no_fit_well, n = lts(model, X_train, y_train, learning_goal)
    acceptable, eps, y_pred = check_acceptable(train_loader, model, learning_goal)
    if acceptable:
        pass        
    else:
        torch.save(model, "unacceptable/selecting_wt.pth")
        # store model in acceptable/wt.pth if acceptable
        acceptable, model, train_loss_list, test_loss_list = \
            module_weight_EU_LG_UA(model, train_loader, test_loader, **config_wt)
        if acceptable:
            print("////////// Start REORG with accpetable wt //////////")
            # load model in acceptable/wt.pth if wt acceptable
            pre_module = "wt"
            reorg = reorganising(pre_module, train_loader, test_loader, **config_reorg)
            reorg.reorganising()
            model = reorg.model
        else:
            print("////////// Start CRAM and REORG with unacceptable wt")
            # load model in unacceptable/selecting_wt.pth if wt not acceptable
            model = torch.load("unacceptable/selecting_wt.pth")            
            cram = cramming(model, train_loader, X_train, y_train, indices_no_fit_well, **config_cram)
            cram.cram()
            model = cram.model
            acceptable, eps, y_pred = check_acceptable(train_loader, model, learning_goal)
            if acceptable:
                pass
            else:
                print("weird cram")
                print(eps)
                break

            pre_module = "cram"
            reorg = reorganising(pre_module, train_loader, test_loader, **config_reorg)
            reorg.reorganising()
            model = reorg.model
            

    

Total obtaining n: 64
obtaining n over lr goal: 0
tensor([  9, 330, 352, 110, 290, 302, 323, 319,  85, 214, 192, 271, 325, 332,
        372, 236, 324, 322, 333, 102, 199,  92, 114, 139, 131, 358,  73, 112,
        288, 137, 357, 212, 148, 165, 184, 237,  75,  97, 362, 159, 101, 268,
         33,  16, 318, 147, 227, 141, 142, 127,  98, 215, 373,  19, 218, 138,
        155, 213, 361, 111,   3, 140,  11,  21,  44])
Total select n: 65
select n over lr goal: 1
--------- module_EU_LG Epoch 0 ---------
train_loss: 0.0560337562734882
Save model and lr increase


c:\Users\USER\anaconda3\envs\learningalgo\lib\site-packages\torch\nn\modules\loss.py:535: UserWarning: Using a target size (torch.Size([30])) that is different to the input size (torch.Size([30, 1])). This will likely lead to incorrect results due to broadcasting. Please ensure they have the same size.
  return F.mse_loss(input, target, reduction=self.reduction)
c:\Users\USER\anaconda3\envs\learningalgo\lib\site-packages\torch\nn\modules\loss.py:535: UserWarning: Using a target size (torch.Size([5])) that is different to the input size (torch.Size([5, 1])). This will likely lead to incorrect results due to broadcasting. Please ensure they have the same size.
  return F.mse_loss(input, target, reduction=self.reduction)


--------- module_EU_LG Epoch 1 ---------
train_loss: 0.10778767118851344
Restore model and lr decrease
--------- module_EU_LG Epoch 2 ---------
train_loss: 0.05327459673086802
Save model and lr increase
--------- module_EU_LG Epoch 3 ---------
train_loss: 0.08849327452480793
Restore model and lr decrease
--------- module_EU_LG Epoch 4 ---------
train_loss: 0.042767499263087906
Save model and lr increase
--------- module_EU_LG Epoch 5 ---------
train_loss: 0.06665547750890255
Restore model and lr decrease
--------- module_EU_LG Epoch 6 ---------
train_loss: 0.034844087436795235
Save model and lr increase
--------- module_EU_LG Epoch 7 ---------
train_loss: 0.037094175505141415
Restore model and lr decrease
--------- module_EU_LG Epoch 8 ---------
train_loss: 0.03143196149418751
Save model and lr increase
--------- module_EU_LG Epoch 9 ---------
train_loss: 0.03047274798154831
Save model and lr increase
--------- module_EU_LG Epoch 10 ---------
train_loss: 0.029535545967519283
Save model

In [ ]:
model = init_model(X_train, y_train)
train_loader, indices_no_fit_well, n = lts(model, X_train, y_train, learning_goal)
acceptable, eps, y_pred = check_acceptable(train_loader, model, learning_goal)
indices_no_fit_well, eps

In [ ]:
acceptable, model, train_loss_list, test_loss_list = \
            module_weight_EU_LG_UA(model, train_loader, test_loader, **config_wt)
acceptable, eps, y_pred = check_acceptable(train_loader, model, learning_goal)
eps

In [46]:
model = torch.load("unacceptable/selecting_wt.pth")  
acceptable, eps, y_pred = check_acceptable(train_loader, model, learning_goal)
eps[eps>learning_goal]

tensor([0.3093])

In [89]:
from module.Cram import *
config_cram["s"] = .1
model = torch.load("unacceptable/selecting_wt.pth")          
acceptable, eps, y_pred = check_acceptable(train_loader, model, learning_goal)  
print(max(eps - eps.min()), max(eps))
cram = cramming(model, train_loader, X_train, y_train, indices_no_fit_well, **config_cram)
cram.cram()
model = cram.model
acceptable, eps, y_pred = check_acceptable(train_loader, model, learning_goal)  
max(eps - eps.min())

tensor([0.3071]) tensor([0.3093])
cramming sample 44th |0.00% total of 1
tensor(0.0051)- Xk) != 0 and (s - r*(Xc - Xk))*(s + r*(Xc - Xk)) < 0
tensor(0.0076)
tensor(-0.0203)


tensor([0.5777])